# SIZING REFRIGERANT LINES
---

This example is based on the examples in ***TRANE, Air Conditioning Clinic: Refrigerant Piping (June 2011, TRG-TRC006-EN)***.

In [1]:
import warnings
from pathlib import Path
from hvac import Quantity
from hvac.fluids import Fluid, CoolPropWarning
from hvac.refrigerant_piping import (
    CopperTubing,
    RefrigerantCycleInfo,
    SuctionLineSizer,
    DischargeLineSizer,
    LiquidLineSizer
)

In [2]:
warnings.filterwarnings('ignore', category=CoolPropWarning)

Q_ = Quantity
R22 = Fluid('R22')

Get copper tubes from shelf "./copper-tube-ACR-D" (this shelf was created with script `copper_tubing.py` in the same folder):

In [3]:
CopperTubing.db_path = Path("./copper-tube-ACR-D")
copper_tubes = CopperTubing.get_records()

Set refrigeration cycle info:

In [4]:
cycle_info = RefrigerantCycleInfo(
    refrigerant=R22,
    T_evp=Q_(40, 'degF'),
    T_cnd=Q_(125, 'degF'),
    evp_superheat=Q_(12, 'delta_degF'),
    cnd_subcooling=Q_(15, 'delta_degF'),
    cmp_superheat=Q_(70, 'delta_degF')
)

Set the maximum and minimum cooling capacity of the system:

In [5]:
Q_dot_evp_max = Q_(20, 'refrigeration_ton')
Q_dot_evp_min = Q_(10, 'refrigeration_ton')

## Suction Line

In [6]:
suction_line_sizer = SuctionLineSizer(
    copper_tubes=copper_tubes,
    cycle_info=cycle_info,
    Q_dot_evp_max=Q_dot_evp_max,
    Q_dot_evp_min=Q_dot_evp_min,
)

In [7]:
horizontal_section = suction_line_sizer.size(
    L=Q_(55, 'feet'),
    fittings=[
        ('long-radius-elbow', 3),
        ('short-radius-elbow', 3)
    ]
)
if horizontal_section is None:
    print('no suitable copper tube available')
else:
    print(horizontal_section)

2 1/8 | maximum flow velocity = 9.5 m/s | minimum flow velocity = 4.8 m/s | maximum pressure drop = 5.399 kPa (0.300 K)


In [8]:
vertical_riser = suction_line_sizer.size(
    L=Q_(15, 'feet'),
    elevation=Q_(15, 'feet')
)
if vertical_riser is None:
    print('no suitable copper tube available')
else:
    print(vertical_riser)

1 5/8 | maximum flow velocity = 16.5 m/s | minimum flow velocity = 8.3 m/s | maximum pressure drop = 5.936 kPa (0.330 K)


## Discharge Line

In [9]:
discharge_line_sizer = DischargeLineSizer(
    copper_tubes=copper_tubes,
    cycle_info=cycle_info,
    Q_dot_evp_max=Q_dot_evp_max,
    Q_dot_evp_min=Q_dot_evp_min
)

In [10]:
discharge_line = discharge_line_sizer.size(
    L=Q_(50, 'feet'),
    fittings=[
        ('long-radius-elbow', 2)
    ]
)
if discharge_line is None:
    print('no suitable copper tube available')
else:
    print(discharge_line)

1 3/8 | maximum flow velocity = 7.9 m/s | minimum flow velocity = 4.0 m/s | maximum pressure drop = 13.649 kPa (0.297 K)


## Liquid Line

In [11]:
liquid_line_sizer = LiquidLineSizer(
    copper_tubes=copper_tubes,
    cycle_info=cycle_info,
    Q_dot_evp_max=Q_dot_evp_max
)

In [12]:
liquid_line = liquid_line_sizer.size(
    L=Q_(100, 'feet'),
    fittings=[
        ('long-radius-elbow', 4),
        ('ball-valve', 2),
        ('sight-glass', 1)
    ],
    elevation=Q_(4, 'feet')
)
if liquid_line is None:
    print('no suitable copper tube available')
else:
    print(liquid_line)

5/8 | maximum flow velocity = 2.7 m/s | minimum flow velocity = nan m/s | maximum pressure drop = nan kPa (nan K)


Add additional accessories to the liquid line and get the pressure drop along the liquid line:

In [13]:
liquid_line.pressure_drop(dP_add=[
    Q_(6, 'psi'),  # filter drier
    Q_(4, 'psi')   # solenoid valve
])
print(liquid_line)

5/8 | maximum flow velocity = 2.7 m/s | minimum flow velocity = nan m/s | maximum pressure drop = 218.188 kPa (4.965 K)
